# 3.3 텐서플로의 함수와 미분

[https://datascienceschool.net/view-notebook/4b286ba9c76c4b36a9218074c8dce524/](https://datascienceschool.net/view-notebook/4b286ba9c76c4b36a9218074c8dce524/)

In [1]:
# 코랩에서는 다음 코드로 베타버전을 설치
!pip install tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 38kB/s 
     |████████████████████████████████| 3.1MB 38.6MB/s 
     |████████████████████████████████| 501kB 47.7MB/s 


In [2]:
import tensorflow as tf
tf.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

'2.0.0-beta1'

## 변수형 텐서

변수형 텐서는 텐서의 값이 바뀔 수 있다. `Variable` 클래스로 정의하며 항상 초기값을 지정해 주어야 한다. 자료형과 크기는 초기값으로부터 자동으로 유추한다.

In [0]:
# 실수 변수형 텐서
s = tf.Variable(1.0)

In [0]:
# 벡터 변수형 텐서
v = tf.Variable(tf.ones((2,)))

In [0]:
# 행렬 변수형 텐서
x = tf.Variable(tf.ones((2, 1)))

변수 텐서의 값을 바꿀 때는 `assign`, `assign_add`, `assign_sub` 메서드를 사용한다.

- `assign`
  - 값을 완전히 할당
  - `=` 에 해당  

- `assign_add`
  - 값을 증가
  - `+=` 에 해당  

- `assign_sub`
  - 값을 감소
  - `-=`에 해당

In [7]:
# 다음과 같이 하면 안된다! 변수형 텐서가 상수형 텐서로 변한다.
# x = tf.ones((2, 1))
x.assign(tf.ones((2, 1)))
x.numpy()

array([[1.],
       [1.]], dtype=float32)

<br>

## 함수

텐서플로우 함수는 파이토치(PyTorch), 데아노(theano) 등의 딥러닝 라이브러리에 있는 함수 기능을 본따 텐서플로우 버전 2에서 새로 만들어진 방법이다. 함수를 사용하면 텐서플로우 버전 1에서처럼 플레이스홀더(placeholder)와 계산 그래프 등을 명시적으로 사용하지 않고 선언적으로 계산 과정을 구현할 수 있다.

텐서플로우 함수는 일반 파이썬 함수처럼 정의하면 된다.

In [0]:
def f(x) :
  return 5 * x

만약 속도 향상을 위한 컴파일이 가능하도록 하려면 함수에 `@tf.function` 데코레이터를 적용하여 구현한다.

In [0]:
@tf.function
def g(a, b) :
  return tf.matmul(a, b)

<br>

## 미분

변수 텐서 혹은 변수 텐서를 포함하는 연산의 결과로 만들어진 텐서를 입력으로 가지는 함수는 그 변수 텐서로 미분한 값을 계산할 수 있다.

1. `GradientTape()` 로 만들어진 gradient tape 컨텍스트 내에서 함수값 결과를 저장한 텐서 `y`를 만든다.
2. `tape.gradient(y, x)` 명령으로 변수형 텐서 `x`에 대한 `y`의 미분값을 계산한다.

In [10]:
x = tf.Variable(tf.constant(1.0))

with tf.GradientTape() as tape :
  y = tf.multiply(5, x)

gradient = tape.gradient(y, x)
gradient.numpy()

5.0

동시에 여러 변수에 대한 그레디언트 벡터를 구할 수도 있다.

In [11]:
x1 = tf.Variable(tf.constant(1.0))
x2 = tf.Variable(tf.constant(1.0))

with tf.GradientTape() as tape :
  y = tf.multiply(x1, x2)

gradients = tape.gradient(y, [x1, x2])
gradients[0].numpy(), gradients[1].numpy()

(1.0, 1.0)

이 때 미분하는 텐서가 변수가 아니라 상수형이면 결과로는 `None`이 출력된다.

In [13]:
x = tf.Variable(tf.constant(1.0))
a = tf.constant(1.0)

with tf.GradientTape() as tape :
  y = tf.multiply(a, x)

gradient = tape.gradient(y, a)
gradient is None

True

만약 상수형 텐서에 대해 미분하고 싶으면 `taple.watch()` 함수를 사용하여 상수형 텐서를 변수형 텐서처럼 바꿔야 한다.

In [14]:
with tf.GradientTape() as tape :
  tape.watch(a)
  y = tf.multiply(a, x)

gradient = tape.gradient(y, a)
gradient.numpy()

1.0